In [245]:
# from session 1 of advanced predictive modeling

from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import math as math
%matplotlib inline

In [246]:
# https://github.com/magee256/TurbomoleScripts/blob/master/SetUp/add_ligand/add_ligand.py

# add arguments for original coordinates, and both components of transformation_matrix, 
def rotate_frag(coordinates, rots):
    """
    Returns coordinates rotated counterclockwise about 
    a rotation axis, with both degree rotated and axis direction 
    specified by the Euler-Rodrigues formula vector parameters in rots.
    
    Parameters
    ----------
    coordinates: numpy array of shape (N,3)
    rots: numpy array of shape (_,_) , representing coefficients of axis of rotation * sin(theta/2), see formula
    
    Returns
    -------
    Transformed `coordinates` rotated about the rotation axis `rots`
    
    """
    aa = 1 - np.dot(rots,rots)
    print "aa is %f \n" % aa
    a = math.sqrt(aa)
    print "a is %f \n" % a
    bb, cc, dd = rots[0]*rots[0], rots[1]*rots[1], rots[2]*rots[2]
    bc, ad, ac, ab, bd, cd = rots[0]*rots[1], a*rots[2], a*rots[1], \
                             a*rots[0], rots[0]*rots[2], rots[1]*rots[2]
    r_mat = np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                    [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                    [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

    return np.dot(coordinates,r_mat.T)

In [247]:
# load in coordinates
rough=np.array(
[[-2.00861,   2.12329,   2.71423],
 [ -2.13619,   2.02111,   3.52290],
 [ -1.81206,   1.37010,   3.91266],
  [-1.36039,   0.82134,   3.49370],
  [-1.23285,   0.92355,   2.68512],
  [-1.55698,   1.57456,   2.29536],
  [-1.33903,   1.53045,   1.50082],
  [-0.81464,   0.47720,   2.13140],
  [-0.88026,   0.85225,   1.39957],
  [-0.54584,   0.59443,   0.69083],
  [-0.65931,   1.02926,  -0.00106],
  [-1.10720,   1.72192,   0.01579],
  [-0.34231,   0.80587,  -0.72927]])

print "Coordinate dimensions: ", rough.shape,'\n'

Coordinate dimensions:  (13, 3) 



In [248]:
# find center of coordinates
centroid = np.mean(rough,axis=0)

# scale center to origin
scaled = np.add(rough,-centroid)

print "Center of original coordinates: \n", centroid,'\n'
print "Center of scaled coordinates: \n", np.mean(scaled,axis=0),'\n'
print scaled

Center of original coordinates: 
[-1.21505154  1.21887154  1.81785   ] 

Center of scaled coordinates: 
[ -8.54017711e-17  -2.64745490e-16   3.41607085e-17] 

[[-0.79355846  0.90441846  0.89638   ]
 [-0.92113846  0.80223846  1.70505   ]
 [-0.59700846  0.15122846  2.09481   ]
 [-0.14533846 -0.39753154  1.67585   ]
 [-0.01779846 -0.29532154  0.86727   ]
 [-0.34192846  0.35568846  0.47751   ]
 [-0.12397846  0.31157846 -0.31703   ]
 [ 0.40041154 -0.74167154  0.31355   ]
 [ 0.33479154 -0.36662154 -0.41828   ]
 [ 0.66921154 -0.62444154 -1.12702   ]
 [ 0.55574154 -0.18961154 -1.81891   ]
 [ 0.10785154  0.50304846 -1.80206   ]
 [ 0.87274154 -0.41300154 -2.54712   ]]


In [249]:
from sklearn.decomposition import PCA

# initialize and train PCA model on original coordinates
# get top 2 principal components of the coordinate matrix
pca = PCA(n_components=2)
reduced = pca.fit_transform(scaled)

# get matrix of the pca eigenvectors
transformation_mat = pca.components_
print "Transformation matrix: \n", transformation_mat,'\n'
print "Matrix dimensions: ", transformation_mat.shape,'\n'

# check that vectors are normalized
print np.dot(transformation_mat,transformation_mat.T)

Transformation matrix: 
[[ 0.31031726 -0.13361454 -0.94119623]
 [ 0.47152379 -0.83806293  0.27443731]] 

Matrix dimensions:  (2, 3) 

[[  1.00000000e+00  -1.66725494e-16]
 [ -1.66725494e-16   1.00000000e+00]]


In [250]:
# get the angle between two vectors, component 1 from PCA, and x-axis (1,0,0)
# but really we're getting sin(theta/2)
# https://en.wikipedia.org/wiki/Euler%E2%80%93Rodrigues_formula
u=transformation_mat[0,:]
print "component 1: ",u,"\n"
v=np.array([1,0,0])
sinHalfAng = math.sqrt((1-np.dot(u,v))/2)

# get x axis of rotation
xaxis = np.cross(u,v)
xaxis /= math.sqrt(np.dot(xaxis,xaxis)) # normalize
bcd = xaxis * sinHalfAng
print np.dot(bcd,bcd)
print "bcd: ",bcd,"\n"

# perform rotation
scaledx = rotate_frag(scaled, bcd)
tmatx = rotate_frag(transformation_mat, bcd)
print tmatx

component 1:  [ 0.31031726 -0.13361454 -0.94119623] 

0.344841367698
bcd:  [ 0.         -0.58140258  0.08253734] 

aa is 0.655159 

a is 0.809419 

aa is 0.655159 

a is 0.809419 

[[-0.80740639 -0.0829258  -0.58413888]
 [ 0.29264395 -0.91598598 -0.27446166]]


In [251]:
# now do all the same with y-axis (0,1,0)

# get rotation angle
u=transformation_mat[1,:]
#u=tmatx[1,:]
print "component 2: ",u,"\n"
v=np.array([0,1,0])
sinHalfAng = math.sqrt((1-np.dot(u,v))/2)

# get y axis of rotation
yaxis = np.cross(u,v)
yaxis /= math.sqrt(np.dot(xaxis,xaxis)) # normalize
bcd = yaxis * sinHalfAng
print np.dot(bcd,bcd)
print "bcd: ",bcd,"\n"

# perform rotation
scaledxy = rotate_frag(scaledx, bcd)
tmatxy = rotate_frag(tmatx, bcd)
print tmatxy

component 2:  [ 0.47152379 -0.83806293  0.27443731] 

0.273550196655
bcd:  [-0.26309243  0.          0.45203161] 

aa is 0.726450 

a is 0.852320 

aa is 0.726450 

a is 0.852320 

[[-0.40240742  0.84656449 -0.34842049]
 [-0.46748239 -0.51725687 -0.71687206]]


In [252]:
print scaled.shape
print scaled,'\n'
print scaledx,'\n'
print scaledxy,'\n'

(13, 3)
[[-0.79355846  0.90441846  0.89638   ]
 [-0.92113846  0.80223846  1.70505   ]
 [-0.59700846  0.15122846  2.09481   ]
 [-0.14533846 -0.39753154  1.67585   ]
 [-0.01779846 -0.29532154  0.86727   ]
 [-0.34192846  0.35568846  0.47751   ]
 [-0.12397846  0.31157846 -0.31703   ]
 [ 0.40041154 -0.74167154  0.31355   ]
 [ 0.33479154 -0.36662154 -0.41828   ]
 [ 0.66921154 -0.62444154 -1.12702   ]
 [ 0.55574154 -0.18961154 -1.81891   ]
 [ 0.10785154  0.50304846 -1.80206   ]
 [ 0.87274154 -0.41300154 -2.54712   ]] 

[[ 0.71825804  0.91209693  0.95046802]
 [ 1.42613219  0.75074368  1.3423148 ]
 [ 1.80657157  0.02788794  1.22598512]
 [ 1.47908668 -0.53353536  0.71782339]
 [ 0.77128884 -0.37215581  0.32604054]
 [ 0.39084946  0.35069993  0.44237022]
 [-0.29522869  0.35432549 -0.01591499]
 [ 0.32026859 -0.81516011 -0.20411198]
 [-0.33877793 -0.36621504 -0.41541659]
 [-0.93651355 -0.59718445 -0.93501791]
 [-1.56482991 -0.08671365 -1.09408541]
 [-1.5954093   0.65473641 -0.73355254]
 [-2.18169599 